In [25]:
import pandas as pd
import calendar
import os
from datetime import datetime

In [45]:
# Read the excel file, excluding the metadata at the beginning, and totals at the end
# Also drop all rows that are NaN
sheet = pd.read_excel("data/april_2017_pollen.xls", skiprows=4, skip_footer=1).dropna(axis=0, how='all')
sheet.head()

,DATE,Ash,Ashe Juniper / Bald Cypress,Black Gum,Walnut,Bushes,Birch,Cotton Wood,Dogwood,Elm,...,Ragweed,Rumex,Sagebrush,Saltbrush,Sedge,Sneezeweed,Other weed pollen,Weed Total,POLLEN TOTAL,Tech.
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Weekend
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Weekend
2,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,8.0,NaN,6.0,18.0,365.0,RP
3,4.0,8.0,NaN,NaN,2.0,NaN,NaN,14.0,NaN,2.0,...,NaN,12.0,2.0,NaN,4.0,NaN,8.0,30.0,209.0,RP
4,5.0,12.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,6.0,16.0,160.0,RP


In [44]:
from download_data import all_data
file_format = os.path.join("data", "{}_{}_{}.xls")
files = [file_format.format(m, y, t) for m, y, t in all_data if os.path.exists(file_format.format(m, y, t))]

In [41]:
sheets = [pd.read_excel(file, skiprows=4, skip_footer=1).dropna(axis=0, how='all') for file in files]